In [1]:
import time
import pandas as pd
import requests as re
from bs4 import BeautifulSoup as bs


In [2]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
"HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
"MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
"NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
"SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [3]:
t1 = time.time()
df = pd.DataFrame(columns = ['park_state','park_name','park_type', 'park_loc', 'park_desc'])
for state in states:
    url = "https://www.nps.gov/state/%s/index.htm" % state
    result = re.get(url)
    soup = bs(result.content,'lxml')
    parks = soup.find_all('li','clearfix')
    for park in parks[:-1]:
        park_name = park.find('h3').text
        park_type = park.find('h2').text
        park_loc = park.find('h4').text
        park_desc = park.find('p').text

        df.loc[len(df)] =  [state, park_name, park_type, park_loc, park_desc]
print("Time to retrieve: %0.3f seconds" % (time.time() - t1))

Time to retrieve: 16.510 seconds


In [4]:
df.head()

,park_state,park_name,park_type,park_loc,park_desc
0,AL,Birmingham Civil Rights,National Monument,AL,"\nIn 1963, images of snarling police dogs unle..."
1,AL,Freedom Riders,National Monument,AL,"\nIn 1961, a small interracial band of “Freedo..."
2,AL,Horseshoe Bend,National Military Park,"Daviston, AL","\nOn 27 March 1814, Major General Andrew Jacks..."
3,AL,Little River Canyon,National Preserve,"Fort Payne, AL",\nLittle River is unique because it flows for ...
4,AL,Muscle Shoals,National Heritage Area,,\nThe Tennessee River brought the early Native...


In [5]:
df['park_lat'] = ''
df['park_long'] = ''

In [6]:
df.head()

,park_state,park_name,park_type,park_loc,park_desc,park_lat,park_long
0,AL,Birmingham Civil Rights,National Monument,AL,"\nIn 1963, images of snarling police dogs unle...",,
1,AL,Freedom Riders,National Monument,AL,"\nIn 1961, a small interracial band of “Freedo...",,
2,AL,Horseshoe Bend,National Military Park,"Daviston, AL","\nOn 27 March 1814, Major General Andrew Jacks...",,
3,AL,Little River Canyon,National Preserve,"Fort Payne, AL",\nLittle River is unique because it flows for ...,,
4,AL,Muscle Shoals,National Heritage Area,,\nThe Tennessee River brought the early Native...,,


In [7]:
t1 = time.time()
for i in range(0,len(df)-1):
    park_search = (df.park_name[i] + ' ' + df.park_type[i]).replace(' ','+')
    park_search += '+coordinates'

    url = "https://www.google.com/search?q=%s" % park_search
    result = re.get(url)
    soup = bs(result.content,"lxml")
    
    try:
        spans = soup.find_all('span', attrs={'class':'mrH1y'})
        park_coords = spans[0].string

        park_coords =park_coords.replace(" ","").split(',')
        lat, long = park_coords[0],park_coords[1]
        lat = lat.split("°")
        long = long.split("°")

        park_lat= float(lat[0])
        if(lat[1] == 'S'):
            park_lat = park_lat * -1

        park_long = float(long[0])
        if(long[1] == 'W'):
            park_long = park_long * -1

        df.park_lat[i] = park_lat
        df.park_long[i] = park_long
    except:
        pass

print("Time to retrieve: %0.3f seconds" % (time.time() - t1))

KeyboardInterrupt: 

In [ ]:
df.head()

In [ ]:
# df.to_csv('df.to_excel("C:\\users\\Owner\\desktop\\df.csv")')